✍ Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов JOIN.

А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

> Обратите внимание! Вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.

Но прежде, как всегда, разберёмся с датасетом

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах `city`, `customer`, `driver`, `shipment`, `truck`. Давайте внимательно их изучим.

Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

<center> <img src='img/dst3-u2-md4_1_1.jpg' width="50%" height="50%"> </center>

Таблица `city` — это справочник городов. Структура справочника представлена ниже.

<center> <img src='img/city.png' width="50%" height="50%"> </center>

Таблица `customer` — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

<center> <img src='img/customer.png' width="50%" height="50%"> </center>

Следующая таблица — `driver` — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

<center> <img src='img/driver.png' width="50%" height="50%"> </center>

В таблице `truck` хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

<center> <img src='img/truck.png' width="50%" height="50%"> </center>

Последняя таблица в датасете, `shipment`, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

<center> <img src='img/shipment.png' width="50%" height="50%"> </center>

Cмоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

```SQL
SELECT          book_name object_name, 'книга' object_description 
FROM          public.books
UNION ALL
SELECT          movie_title, 'фильм' 
FROM          sql.kinopoisk
```

<center> <img src='img/dst3-u2-md4_2_1.png' width="50%" height="50%"> </center>

В запросе мы использовали оператор `UNION ALL` — он присоединяет любой результат запроса к другому *«снизу»* при условии, что у них одинаковая структура, а именно:

> одинаковый тип данных;
> одинаковое количество столбцов;
> одинаковый порядок столбцов согласно типу данных.

### Виды UNION

Оператор присоединения существует в двух вариантах:

- `UNION` выводит только уникальные записи;
- `UNION ALL` присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.
  
> Важно! `UNION` оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в 
> данном случае можно провести аналогию с `DISTINCT`). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам 
> не важны, предпочтительнее использовать `UNION ALL`.

Проверить, как различаются операторы, вы сможете при выполнении заданий 2.1 и 3.1.

Синтаксис

Запрос строится таким образом:

```SQL
SELECT         n columns
FROM 
         table_1
UNION ALL
SELECT 
         n columns
FROM 
         table_2
...
UNION ALL
SELECT 
         n columns
FROM 
         table_n
```

Результатом выполнения такого запроса будут строки `table_1`, `table_2`, ..., `table_n`, соединённые одни под другими и выведенные в единой выдаче.

> Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке `SELECT`, даже если они отличаются в других 
> блоках подзапросов.

Пришла пора испытать функцию `UNION(ALL)` на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с `ID` всех таблиц и указанием объекта, к которому он относится.

```SQL
SELECT
         c.city_id object_name, 'id города' object_type
FROM 
         sql.city c
UNION ALL
SELECT
         d.driver_id other_name, 'id водителя' other_type
FROM 
         sql.driver d
UNION ALL
SELECT
         s.ship_id, 'id доставки'
FROM 
         sql.shipment s
UNION ALL
SELECT
         c.cust_id, 'id клиента'
FROM 
         sql.customer c
UNION ALL
SELECT
         t.truck_id, 'id грузовика'
FROM 
         sql.truck t
ORDER BY 1
```

> Обратите внимание! Несмотря на исходные названия колонок `other_name` и `other_type` во втором подзапросе, в выводе мы 
> получим названия, которые дали в первом блоке: object_name и object_type.

> Другая особенность — в применении сортировки `ORDER BY`: она всегда будет относиться к итоговому результату всего запроса 
> с `UNION ALL`.


В случаях, когда необходимо применить команду `ORDER BY` или `LIMIT` не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы `ORDER BY` и `LIMIT` ко всему результату запроса.

```SQL
    SELECT book_name object_name, 'книга' object_descritption 
    FROM public.books
UNION ALL
    SELECT movie_title, 'фильм' 
    FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
```

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим `ORDER BY` и `LIMIT` ещё и в первую часть запроса:

```SQL
    SELECT book_name object_name, 'книга' object_descritption 
    FROM public.books
    ORDER BY 1
    LIMIT 1
UNION ALL
    SELECT movie_title, 'фильм' 
    FROM sql.kinopoisk
    ORDER BY 1
    LIMIT 1
```

Вместо результата получим сообщение о синтаксической ошибке: "`...syntax error at or near "UNION"...`" Очевидно, этот фокус не удался.

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

```SQL
    (SELECT book_name object_name, 'книга' object_descritption 
    FROM public.books
    ORDER BY 1
    LIMIT 1)
UNION ALL
    (SELECT movie_title, 'фильм' 
    FROM sql.kinopoisk
    ORDER BY 1
    LIMIT 1)
```

Отлично! Мы получили именно то, что хотели.

#### Почему так важен тип данных?

Как мы уже знаем, `UNION` может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех `id` городов и их названий в одном столбце.

Давайте напишем запрос, который позволит получить нужный нам результат.

```SQL
    SELECT 
            c.city_id
    FROM
            sql.city c
UNION ALL
    SELECT 
            cc.city_name
    FROM
            sql.city cc
```

Вместо результата вы получите сообщение об ошибке: "`ERROR: UNION types integer and text cannot be matched`". Дело в том, что мы попытались объединить **числовой** и **строковый** типы в одной колонке, а это невозможно.

Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в `Postgres` составляется запрос по модели `column_name::column_type`.

Таким образом, чтобы перевести `city_id` в текст, нам потребуется написать `city_id::text`.

> Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для 
> соединения разнородных сущностей. Главное — помнить, что сортировка *текста* отличается от сортировки *чисел* и *дат*.

Немного подправим запрос, чтобы получить желаемый результат.

```SQL
    SELECT 
            c.city_id::text
    FROM
            sql.city c
UNION ALL
    SELECT 
            cc.city_name
    FROM
            sql.city cc
```

### Возможности UNION

✍ Помимо соединения разнородных сущностей в единый справочник, `UNION ALL` часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с `UNION` могут использоваться функции группировки и выборки.

Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.

```SQL
SELECT
         c.city_name,
         c.population
FROM
         sql.city c
UNION ALL
SELECT
         'total',
         SUM(c.population)
FROM
         sql.city c
ORDER BY 2 DESC
```

Визуально это действие можно представить так:

<center> <img src='img/dst3-u2-md4_4_1.png' width="50%" height="50%"> </center>

✍ `UNION` также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

Например, с помощью `UNION` можно отобразить, у кого из водителей заполнен столбец с номером телефона.

```SQL
    SELECT  d.first_name,
            d.last_name,
            'телефон заполнен' phone_info
    FROM
            sql.driver d
    WHERE d.phone IS NOT NULL
UNION
    SELECT d.first_name,
            d.last_name,
            'телефон не заполнен' phone_info
    FROM
            sql.driver d
    WHERE d.phone IS NULL
```

✍ `UNION` можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

```SQL
SELECT 
         'a' letter,'1' ordinal_position
UNION 
SELECT 
         'b','2'
UNION 
SELECT
         'c','3'
```

Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.